<a href="https://colab.research.google.com/github/Asylum0741/Smart-LMS/blob/main/Finetuning_TinyLlama_LLM_Smart_LMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft accelerate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, get_peft_model
import torch
from datasets import load_dataset

# Load the TinyLlama base model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move model to GPU (Colab provides free GPUs)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


In [ ]:
from datasets import load_dataset

# Load the dataset from a JSONL file (replace the path with your actual file path)
dataset = load_dataset('json', data_files='training_dataset.jsonl', split='train')


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the raw dataset (Assuming it's in a JSONL format)
dataset = load_dataset("json", data_files="training_dataset.jsonl", split="train")

# Load tokenizer for TinyLlama
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Function to format and tokenize the dataset
def format_example(examples):
    # Tokenize inputs and outputs
    model_inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=512)

    # Add the 'labels' key for language model
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the formatting function to the dataset
tokenized_dataset = dataset.map(format_example, batched=True, num_proc=4)


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments
import torch

In [ ]:
from transformers import AutoModelForCausalLM

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,   # Rank for LoRA
    lora_alpha=32,  # Scaling factor for LoRA
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
)

# Load the TinyLlama model
model = AutoModelForCausalLM.from_pretrained(model_name)
model.gradient_checkpointing_enable()
# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

# Move the model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
lora_model.to(device)


In [ ]:
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    fp16=True,  # Use mixed precision training
    save_steps=500,
    logging_dir="./logs",
)



trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Train dataset
    tokenizer=tokenizer,
)

# Start fine-tuning
trainer.train()


In [ ]:
# Save the model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")


In [ ]:
!zip -r finetunemodel.zip ./trained_model

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive', force_remount=False)


In [ ]:
!cp ./trained_model/model.safetensors /content/gdrive/MyDrive/
